In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mizani.formatters import percent_format
import os
from plotnine import *
import numpy as np
import sys
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse

In [11]:
# Read the csv and feed it into the variable xc
data = pd.read_csv("morg-2014-emp.csv")

# Filter dataset for healthcare practitioners
data = xc.loc[((xc["occ2012"] >= 3000) & (xc["occ2012"] <= 3540))]

In [12]:
data.head()

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
10,25,620910090003769,January,AL,3851.5450,1800.00,40,43,1,NaN,...,2,4,"Native, Born In US",63,Pharmacies and drug stores (44611),3050,"Private, For Profit",No,No,Employed-At Work
33,60,7087505071191,January,AL,3712.0546,1171.60,40,41,1,NaN,...,0,0,"Native, Born In US",63,Business support services (5614),3510,"Private, For Profit",No,No,Employed-At Work
37,77,97953206049188,January,AL,3414.1067,1730.76,60,44,1,NaN,...,0,0,"Native, Born In US",63,Home health care services (6216),3255,"Private, For Profit",No,No,Employed-Absent
42,89,130003560100156,January,AL,3296.7986,2884.61,60,45,1,NaN,...,2,10,"Native, Born In US",63,Offices of physicians (6211),3060,"Private, For Profit",No,No,Employed-At Work
71,166,42510310991895,January,AL,3322.0487,618.75,45,40,1,NaN,...,0,0,"Native, Born In US",63,Hospitals (622),3320,"Private, For Profit",No,No,Employed-At Work


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9208 entries, 10 to 149315
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  9208 non-null   int64  
 1   hhid        9208 non-null   int64  
 2   intmonth    9208 non-null   object 
 3   stfips      9208 non-null   object 
 4   weight      9208 non-null   float64
 5   earnwke     9208 non-null   float64
 6   uhours      9208 non-null   int64  
 7   grade92     9208 non-null   int64  
 8   race        9208 non-null   int64  
 9   ethnic      586 non-null    float64
 10  age         9208 non-null   int64  
 11  sex         9208 non-null   int64  
 12  marital     9208 non-null   int64  
 13  ownchild    9208 non-null   int64  
 14  chldpres    9208 non-null   int64  
 15  prcitshp    9208 non-null   object 
 16  state       9208 non-null   object 
 17  ind02       9208 non-null   object 
 18  occ2012     9208 non-null   int64  
 19  class       9208 non-null   o

### EDA

In [14]:
data["sex"] = data["sex"].fillna("Missing")
data["earnwke"] = data["earnwke"].fillna("Missing")
data["uhours"] = data["uhours"].fillna("Missing")
data["lfsr94"] = data["lfsr94"].fillna("Missing")
data["marital"] = data["marital"].fillna("Missing")
data["grade92"] = data["grade92"].fillna("Missing")


In [20]:
data["female"] = (data["sex"] == 2)
data["w"] = data["earnwke"] / data["uhours"]
data["lnw"] = np.log(data["w"]) 
data["working"] = (data["lfsr94"] == 1)
data["married"] = (data["marital"] != 7)
data['grade92'] = pd.to_numeric(data['grade92'], errors='coerce')
data["university"] = (data["grade92"] > 40) 

# Convert to numeric
data['w'] = pd.to_numeric(data['w'], errors='coerce')
data["female"].value_counts()


female
True     7220
False    1988
Name: count, dtype: int64

In [22]:
data.shape

(9208, 29)

In [25]:
# List of columns to keep
columns_to_keep = ["female", "lnw", "working", "married", "university"]
# Drop all other columns
data = data[columns_to_keep]
data.shape

(9208, 5)

In [26]:
data

,female,lnw,working,married,university
10,True,3.806662,False,True,True
33,True,3.377246,False,True,True
37,True,3.361971,False,True,True
42,False,3.872800,False,True,True
71,True,2.621039,False,True,False
...,...,...,...,...,...
149201,True,4.278266,False,True,True
149242,True,3.178054,False,True,True
149252,True,2.088773,False,False,False
149290,True,2.397895,False,True,True
